In [1]:
import pdfplumber

# path = "~/Documents/GitHub/Order-Assistant/StoreReports-1.pdf"
path = 'StoreReports-1.pdf'

# all files are formatted the same
PAGES = 4

In [2]:
with pdfplumber.open(path) as pdf:
    page = pdf.pages[PAGES]
    text = page.extract_text()

In [20]:
# list of all items 
drinks = [
    '591ml Pepsi',
    '591ml Diet Pepsi',
    '591ml Mountain Dew',
    '591ml Diet Mtn Dew', 
    '591ml Aquafina', 
    '591ml Dr Pepper', 
    '591ml Mug Root Beer', 
    '591ml Schw Gngr Ale', 
    '591ml WCherry Pepsi',
    '591ml Crush Orange',
    '591ml Seven Up',
    '591ml Brsk IceT',
    '2Ltr Pepsi',
    '2Ltr Diet Pepsi',
    '2Ltr Mountain Dew',
    '2Ltr Dt Mtn Dew',
    '2Ltr Dr Pepper',
    '2Ltr Mug Root Beer',
    '2Ltr Schw Gngr Ale',
    '2Ltr Crush Orange',
    '2Ltr Brsk SwTea Lm',
    '2Ltr Seven U '
    '355ml Pepsi',
    '355ml Diet Pepsi',
    '355ml Diet Mtn Dew',
    '355ml Dr Pepper',
    '355ml Schw Gngr Ale',
    '355ml Brsk SwTea Lm',
    '355ml Seven Up',
    '12oz Welch Gr Soda'
]

cheese = [
    'Cheddar Cheese',
    'Three Cheese Blend',
    'String Cheese',
    'Agropur Cheese',
    '2 Cheese',
    '3 Cheese',
    'Feta Cheese',
    'Philly Whipped Cream Chee'

]

dough = [
    'Dough Tray 10',
    'Dough Tray 12',
    'Dough Tray 14',
    'Dough Tray 16',
    'Chocolate Chip Cookie',
    'Choc Chip Brownie',
    'Cinnamon Pull Apart',
    'Thin Crust 14',
    'Gluten Free Crust'
]

meats = [
    'Pepperoni',
    'Bacon',
    'Salami',
    'Italian Sausage',
    'Anchovies',
    'Grilled Chicken',
    'Chicken Popper',
    'Salami',
    'Canadian Bacon',
    'Philly Cheesesteak',
    'Sliced Spicy Meatballs',
    'Meatball',
    'Peperoni',
    'Beef',
    'Sausage',
    'Italian Sausage',
    'Roasted Wings',
    'Donair Topping'
]

produce = [
    'Banana Peppers',
    'Black Olives',
    'Pineapple Lg Can',
    'Mushrooms',
    'Fresh Spinach',
    'Green Peppers',
    'Onions',
    'Tomatoes',
    'Banana Peppers',
    'Pepperoncini Peppers',
    'Jalapeno Peppers',
    'Green Olives'
]

sauces = [
    'Bulk Ranch Sauce',
    'Pizza Sauce',
    'Jug Garlic Sauce',
    'Garlic Sauce Cups',
    'Pizza Sauce Cups',
    'Cheese Sauce Cups',
    'Ranch Sauce Cups',
    'Barbecue Sauce Cups',
    'Buffalo Sauce Cups',
    'Honey Must Sauce Cups',
    'Blue Cheese Sauce Cups',
    'Barbecue Sauce',
    'Garlic Parmesan Sauce Jug',
    'Buffalo Sauce',
    'Pouch Honey Chptl',
    'BBQ Bulk',
    'Alfredo Sauce',
    'Spinach Alfredo Sauce',
    'Cream Cheese Flavored Ici',
    'Creamy Garlic Dipping Cup',
    'Donair Sauce',
    'Hellman\'s Bulk Ranch'
]

In [ ]:
category_dict = {
    'drinks' : drinks,
    'cheese' : cheese,
    'meats' : meats,
    'produce' : produce,
    'sauces' : sauces
}

In [ ]:
# pulls data from page
def pageSearch(page):
    word = ''
    space_counter = 0
    word_counter = 0
    for character in page:
        # 160 is the hex value for ' '
        if ord(character) != 160:
            word += character
            space_counter = 0
        elif len(word) > 0: # word ends if there are more than one space between words
            word += ' '
            space_counter += 1

            if space_counter == 2:
                space_counter = 0                   
                word = ''
                if wordSearch(word):
                    return True

    if word_counter == len(sauces):
        print('Successfully ran')

In [ ]:
# look through the dictionary to find out which category to pull data
def categorySearch(word):
    if word.strip() in category_dict:
        return category_dict[word]
    # check if the word is a new category

def wordSearch(word):
    # check if the word belongs to the current category
    if word in sauces:
        print(word)
        return True
        # check if the word is in the list

In [ ]:
print(text)

09/19/2021                                           PAPA JOHNS PIZZA ­ RESTAURANT 11532                                         Page 3 of 4
04:04:30 pm                                      Actual Inventory Cost Report ­ Detailed View                                       INVCOSRP
                                                           09/13/2021 ­ 09/19/2021
    Item # Description                        Begin  Deliv./Xfr        End       Used         Cost      %  of Net
    SAUCES
    1002   Bulk Ranch Sauce                    0.00        0.00       0.00       0.00         0.00           0.00
    1006   Pizza Sauce                         5.00       48.00       0.00      53.00       240.91           1.53
    1104   Jug Garlic Sauce                    3.10        2.00       0.00       5.10         3.75           0.02
    1105   Garlic Sauce Cups                   5.50        6.00       0.00      11.50       141.09           0.89
    1112   Pizza Sauce Cups                    0.00   